In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import transformers
import torch
from dotenv import load_dotenv
import os
import openai
import banking  # noqa: E402
from private_prompting import Prompter
load_dotenv(".env")

open_ai_key = os.environ.get("openai-key")


Download data and initialize a DuckDB instance. 

In [2]:
_ = banking.BankingData("https://tinyurl.com/jb-bank", "bank")
_.extract_to_csv()

# Loading in SQL extension
%reload_ext sql
# Initiating a DuckDB database named 'bank.duck.db' to run our SQL queries on
%sql duckdb:///banco.duck.db

Create table.

In [3]:
%sql CREATE OR REPLACE TABLE bank AS FROM read_csv_auto('bank_cleaned.csv', header=True, sep=',')

Running query in 'duckdb:///banco.duck.db'

Count
4521


In [4]:
# Extract column names
columns = %sql PRAGMA table_info('bank');
column_names = [row[1] for row in columns]

Running query in 'duckdb:///banco.duck.db'

<h1 align='center'>Prompts (indicaciones)  & Agentes</h1>

<h3 align='center'>Cómo incorporar Prompts en tus scripts de Python y expandir su funcionalidad a través de los agentes</h3>

<h4 align='center'>Laura Funderburk</h4>



<h2 align='center'>Sobre mí</h2>

* Developer Advocate @ Ploomber (hablando y compartiendo conocimientos sobre herramientas para mejorar el flujo de trabajo de la ciencia de datos)

* Anteriormente científica de datos (sector con fines de lucro, sector sin fines de lucro)

* Profundamente curiosa acerca de la IA generativa, Modelos de Lenguaje Grande, con un enfoque en ingeniería y automatización

* Utilizo LLMs, indicaciones y agentes para automatizar tareas de trabajo

<h2 align='center'>Resumen de la charla</h2>

<h3 align='center'>Parte I: Indicaciones (40 minutos)</h3>

1. Casos de uso y tareas de LLMs
2. El ciclo de vida del proyecto de IA Generativa
3. Elegir la arquitectura de LLM correcta
4. Elementos clave de las indicaciones y técnicas de indicación
5. Indicaciones para LLMs privados (API de OpenAI): ChatCompletion
6. Indicaciones para LLMs de código abierto a través de HuggingFace


<h2 align='center'>Resumen de la charla</h2>

<h3 align='center'>Parte II: Agentes y marcos de trabajo de código abierto (20 minutos)</h3>

1. ¿Qué son los agentes?
2. Introducción a Haystack
3. Introducción a LangChain
4. Técnicas para combinar indicaciones y agentes para el despliegue de aplicaciones
5. Pros y contras de cada uno

<h1 align='center'>Parte I: Indicaciones</h1>

<h2 align='center'>Casos de uso y tareas de LLMs</h2>

* Resumen de texto

* Conversación

* Traducción

* Generación de texto

* Clasificación de texto, tokens y sentimientos

* Preguntas y respuestas de tablas y de datos no estructurados

* Similitud de frases

* Enmascaramiento

<h2 align='center'>Casos de uso y tareas de LLMs</h2>

<h3 align='center'>Tu objetivo es entender el caso de negocio que estás resolviendo, y luego seleccionar los métodos apropiados para resolverlo</h3>

$\Rightarrow$ ¿Quién se beneficiará de tu producto?

$\Rightarrow$ ¿Cuáles son las restricciones empresariales (tiempo, datos, recursos)?

$\Rightarrow$ ¿Cuál es el resultado final?

$\Rightarrow$ ¿Cómo será servido?


<h2 align='center'>El ciclo de vida del proyecto de IA generativa</h2>

<p></p>

<center>
  <img src="diagrams/genai_project_lifecycle.jpg" width="1200px"/>

</center>

Source: Coursera, Generative AI with LLMs

<h2 align='center'>Enfoque de esta charla</h2>

<p></p>
<center>
  <img src="diagrams/genai_project_lifecycle_focus.jpg" width="1200px"/>

</center>

<h2 align='center'>Elegir la arquitectura de LLM correcta</h2>

<p></p>
<center>
  <img src="diagrams/opt.jpeg" width="200px"/>

</center>


* Transformadores solo de decodificador: Bueno para **tareas generativas** (auto-regresivas)
* Transformadores solo de codificador: Bueno para tareas que requieren **entender la entrada de datos** (auto-codificación)
* Transformadores de codificador-decodificador o modelos de secuencia a secuencia: Bueno para **tareas generativas que requieren entrada de datos**






<h2 align='center'>Elegir la arquitectura de LLM correcta </h2>

<p></p>

|Tipo de Transformador	| Arquitectura	| Modelo similar	| Enfoque	|Ejemplo|
|-|-|-|-|-|
| Auto-regresivo	| Solo decodificador	| Tipo GPT	| Tareas generativas	|Chat bot |
| Auto-codificación	| Solo codificador	| Tipo BERT	| Comprensión de la entrada	de datos| Respuesta a preguntas|
| Secuencia-a-Secuencia	| Codificador-decodificador|	Tipo BART/T5	| Tareas generativas que requieren una entrada	| Traducción de idiomas|


<p></p>

[https://github.com/christianversloot/machine-learning-articles](https://github.com/christianversloot/machine-learning-articles/blob/main/differences-between-autoregressive-autoencoding-and-sequence-to-sequence-models-in-machine-learning.md)



<h2 align='center'>¿Necesito entrenar un nuevo modelo para resolver mi problema?</h2>

**No. Entrenar un LLM es costoso (uso de GPU, tiempo, cálculo, datos). Por eso, compartir LLMs y sus componentes afinados se ha vuelto muy popular.**


<p></p>
<center>
  <img src="diagrams/hftasks.png" width="1200px"/>

</center>

Puedes comenzar indicando a un LLM, luego afinando* si no estás obteniendo los resultados que deseas. Necesitarás curar un conjunto de datos para esto.

*(ajuste de instrucciones o, por ejemplo, PEFT + LoRA)

Fuente: https://huggingface.co/tasks

<h1 align='center'>Prompting (Indicaciones)</h1>


<h2 align='center'>Elementos clave de las indicaciones</h2>

<h3 align='center'>Básico</h3>

* Un LLM con el que interactuar
* Temperatura
* Tokens máximos
* Una solicitud en lenguaje natural

<h3 align='center'>Avanzado</h3>

* Datos (archivos de texto, archivos web)
* Un sistema de almacenamiento de base de datos (DB vectorial, SQL, PostgreSQL, etc)
* Interfaces de usuario


<h2 align='center'>Técnicas de indicación</h2>

* Inferencia de cero disparos

* Inferencia de un solo disparo

* Inferencia de pocos disparos

* Cadena de pensamiento

* Roles (API de OpenAI)

<h2 align='center'>Indicaciones para LLMs privados (API de OpenAI)</h2>

Vamos a centrarnos en la funcionalidad `ChatCompletion`.

Elementos clave:

* Clave de API de OpenAI
* Modelo elegido (GPT4, GPT 3.5 Turbo, Text-Davinci)
* Temperatura
* Tu indicación


<h3 align='center'>Técnicas de indicación: Inferencia de cero disparos</h3>

**Fórmula: instrucción, sin ejemplos.**

Supongamos que queremos traducir una pregunta en lenguaje natural a SQL.

```python
prompt = f"Responde a la pregunta {natural_question} \
           para la tabla {db_name} \
           con esquema {schema}"
```

<h3 align='center'>Técnicas de indicación: Inferencia de un solo disparo</h3>

**Fórmula: instrucción, un ejemplo.**

```python
prompt = f"Responde a la pregunta {natural_question} \
           para la tabla {db_name} \
           con esquema {schema}\
               Pregunta: ¿Cuántos registros hay?\
               Respuesta: SELECT COUNT(*) FROM bank"
```

<h3 align='center'>Técnicas de indicación: Inferencia de pocos disparos</h3>

**Fórmula: instrucción, más de un ejemplo.**

```python
prompt = f"Responde a la pregunta {natural_question} \
           para la tabla {db_name} \
           con esquema {schema}\
           Pregunta: ¿Cuántos registros hay?\
           Respuesta: SELECT COUNT(*) FROM bank\
           Pregunta: Encuentra todos los empleados que están desempleados\
           Respuesta: SELECT * FROM bank WHERE job = 'unemployed'"

```

<h3 align='center'>Roles en la indicación del punto final de ChatCompletion (solo API de OpenAI)</h3>

El 'rol' puede tomar uno de tres valores: system, user o el assistant

`content` contiene el texto del mensaje del role.

`system` Puedes usar una instrucción a nivel de sistema para guiar el comportamiento de tu modelo a lo largo de la conversación.

`user` ¿Cuáles son las solicitudes típicas que alguien en ese rol recibiría?

`assistant` Este rol representa al modelo de lenguaje, como ChatGPT, que genera respuestas basadas en los mensajes proporcionados por el usuario.

<h2 align='center'>Problema empresarial: traducir preguntas en lenguaje natural a SQL</h2>

Podemos resolver este problema con indicaciones y el punto final ChatCompletion en la API de OpenAI.

**Enfoque: construir una clase Prompter y añadir cada técnica de indicación como un método, luego evaluar los resultados**

<h3 align='center'>Enfoque: inicializar una clase Prompter</h3>

```python
import openai

class Prompter:
    def __init__(self, api_key, gpt_model, temperature=0.2):
        if not api_key:
            raise Exception("Please provide the OpenAI API key")

        self.api_key  = api_key
        self.gpt_model = gpt_model
        self.temperature = temperature
    
    
```
    

<h3 align='center'>Enfoque: añadir un método de completado de chat para llamar a un modelo tipo GPT (API de OpenAI)</h3>

<p></p>
<center>
  <img src="diagrams/init-chatcompletion.png" width="1200px"/>

</center>

<h3 align='center'>Enfoque: añadir un método con una indicación de un solo disparo y el rol de asistente</h3>

<p></p>
<center>
  <img src="diagrams/prompt-roles-assistant.png" width="1200px"/>

</center>


<h3 align='center'>Enfoque: añadir un método con una indicación de un solo disparo y el rol de asistente</h3>

<p></p>
<center>
  <img src="diagrams/prompt-roles-sql.png" width="1200px"/>

</center>


<h2 align='center'>Evaluar resultados</h2>

Supongamos que tengo una instancia en memoria de DuckDB con una tabla llamada bank que se ve así.

In [5]:
%sqlcmd explore --table bank

<h2 align='center'>Evaluar resultados</h2>

Probemos las diferentes técnicas de indicación.

Le pediremos al modelo GPT-3.5-turbo de la API de OpenAI que traduzca una pregunta en lenguaje natural a SQL.

In [6]:
pm  = Prompter(open_ai_key, "gpt-3.5-turbo")

Zero-shot results.

In [16]:
pm.natural_language_zero_shot("bank", 
                              column_names, 
                              "How many unique jobs are there?")

"To determine the number of unique jobs in the table, we need to analyze the 'job' column. However, without the actual data in the table, it is not possible to provide an exact answer."

In [15]:
pm.natural_language_zero_shot("bank", 
                              column_names, 
                              "What is the total balance for \
                               employees by education?")

"Sorry, but I can't provide the answer you're looking for."

Resultados de un solo disparo.

In [9]:
pm.natural_language_single_shot("bank", 
                                column_names, 
                                "How many unique jobs are there?")

'There are 45211 records in the bank table.'

In [10]:
pm.natural_language_single_shot("bank", 
                                column_names, 
                                "What is the total balance for \
                                employees by education?")

'There are 45211 records in the bank table.'

Resultados basados en roles.

In [11]:
pm.natural_language_with_roles("bank", 
                               column_names, 
                               "How many unique jobs are there?")

'SELECT COUNT(DISTINCT job) FROM bank'

In [12]:
pm.natural_language_with_roles("bank", 
                               column_names, 
                               "What is the total balance for\
                               employees by education?")

'SELECT education, SUM(balance) AS total_balance\nFROM bank\nGROUP BY education'

In [13]:
%%sql
SELECT education, SUM(balance) AS total_balance 
FROM bank GROUP BY education;

Running query in 'duckdb:///banco.duck.db'

education,total_balance
primary,957027
secondary,2759854
tertiary,2396822
unknown,318133


<h3 align='center'>Prompting/Indicaciones para LLMs de código abierto a través de HuggingFace</h3>

Necesitas asegurarte de instalar los módulos correctos a través de pip junto con cualquier módulo especificado en la tarjeta del modelo del LLM.

<p></p>
<center>
  <img src="diagrams/hftasks.png" width="800px"/>

</center>


Ejemplo: https://huggingface.co/microsoft/tapex-base

<h3 align='center'>La realidad de prompting en modelos de código abierto</h3>


<p></p>
<center>
  <img src="diagrams/this-is-fine.jpeg" width="600px"/>

</center>


<h3 align='center'>La realidad de las indicaciones en modelos de código abierto</h3>

* Los modelos alojados en HuggingFace se asemejan a repositorios de GitHub (pero no de una buena manera).
* Necesitarás sentirte cómodo usando las bibliotecas transformers, PyTorch y TensorFlow.
* Necesitarás algo más que solo comodidad con las arquitecturas de transformadores.
* Infierno de dependencias.
* Los resultados de las indicaciones varían entre diferentes modelos.
* La documentación del modelo varía desde inexistente hasta altamente técnica (artículos de investigación).
* Mayor probabilidad de que necesites encontrar el modelo base y afinarlo con tus datos para obtener mejores resultados.


<h3 align='center'>Indicando a un modelo tipo T5 para traducir NL a SQL</h3>

Exploraremos la funcionalidad del modelo afinado tipo T5 `mrm8488/t5-base-finetuned-wikiSQL.`

Recuerda que los modelos tipo T5 son de tipo codificador-decodificador y son buenos para traducir entre idiomas.

Este modelo fue afinado en el conjunto de datos wiki-SQL.


```python
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
```

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

In [ ]:
def get_sql(query):
    input_text = "translate English to SQL: %s </s>" % query
    features = tokenizer([input_text], 
                         return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                           attention_mask=features['attention_mask'],
                            max_new_tokens=200)

    return tokenizer.decode(output[0])

# Translate
natural_question = "How many entries are there?" 
db_name = "banks"
schema = column_names

prompt = f"{natural_question} \
           for table {db_name} \
           with schema {schema}"

get_sql(prompt)

<h3 align='center'>Cómo guiar tus elecciones</h3>

1. Recuerda tu caso de uso, las restricciones empresariales y quién usará tu aplicación
2. Recuerda los tres modelos base y sus palabras clave
3. Prepárate para la posibilidad de afinar

<p> </p>

|Tipo de Transformador	| Arquitectura	| Modelo similar	| Enfoque	|Ejemplo|
|-|-|-|-|-|
| Auto-regresivo	| Solo decodificador	| Tipo GPT	| Tareas generativas	|Chat bot |
| Auto-codificación	| Solo codificador	| Tipo BERT	| Comprensión de la entrada	de datos| Respuesta a preguntas|
| Secuencia-a-Secuencia	| Codificador-decodificador|	Tipo BART/T5	| Tareas generativas que requieren una entrada	| Traducción de idiomas|


<h1 align='center'>Parte II: Agentes y marcos de trabajo de código abierto</h1>

Ahora dirigiremos nuestra atención a dos marcos de trabajo de código abierto que puedes utilizar para aumentar la funcionalidad de las indicaciones a través de agentes: LangChain y Haystack.


Los marcos de trabajo presentados aquí pueden ser instalados vía pip e importados como módulos en tu script de Python.


<h2 align='center'>¿Qué son los agentes?</h2>

El papel de un agente es empoderar a los LLMs para decidir qué acciones tomar, otorgándoles un cierto grado de autonomía. En términos simples, los agentes son una fusión de cadenas de LLMs (que son secuencias de LLMs) y herramientas.

<h2 align='center'>Introduciendo LangChain</h2>

<center>
  <img src="diagrams/langchain.png" width="300px"/>
</center>

LangChain es un marco de trabajo para desarrollar aplicaciones impulsadas por modelos de lenguaje. Permite aplicaciones que son:

* Conscientes de los datos: conecta un modelo de lenguaje con otras fuentes de datos

* Agentes: permite a un modelo de lenguaje interactuar con su entorno

<h2 align='center'>¿Cómo aborda LangChain los Agentes?</h2>

<p> </p>

<center>
  <img src="diagrams/langchain.jpg" width="1200px"/>

</center>

Con `LangChain` pensamos en términos de **componentes** y **cadenas listas para usar**.


<h2 align='center'>Cómo incorporarlo en tus scripts</h2>

Puedes construir tus funciones personalizadas en Python y utilizar su decorador @tool. Luego de inicializar LangChain junto con el modelo GPT que quieras, puedes pedirle que realice tareas con comandos de lenguaje natural.

**Es hora de un mini-demo.**

Herramientas que se le dieron al agente:

1. Un rastreador web
2. Un indicador basado en GPT con contenido de system y user
3. Instrucciones para resumir la página web y luego escribir una publicación en redes sociales sobre el resumen

<h2 align='center'>Introduciendo Haystack</h2>

<center>
  <img src="diagrams/haystack-ogimage.png" width="500px"/>
</center>

Haystack es un marco de trabajo de código abierto para construir sistemas de búsqueda que funcionan de manera inteligente sobre grandes colecciones de documentos.


Funcionalidad:

* Llama a modelos de código abierto, modelos alojados (Azure, AWS) así como privados (API de OpenAI)
* Construye pipelines de NLP listos para producción con sus herramientas personalizadas
* Maquinaria para el procesamiento de datos no estructurados (texto)
* Aprovecha sus plantillas de indicaciones ([prompt-hub](https://prompthub.deepset.ai/))
* Incorpora Agentes
* Compatibilidad con Vector y DB clásica.
* Lanza a produccion vía REST API

<h2 align='center'>¿Cómo aborda Haystack los Agentes?</h2>

**Nodos:** cada Nodo logra una cosa

**Pipelines:** esta es la estructura estándar de Haystack que puede conectarse a tus datos y realizar en ellos tareas de NLP que tú defines.

**Herramientas:** puedes pensar en una Herramienta como un experto, que es capaz de hacer algo realmente bien.

**Agente:** un componente que está impulsado por un LLM, como GPT-3. Puede utilizar herramientas y decidir el próximo mejor curso de acción para llegar al resultado de una consulta.

<h2 align='center'>Creando un nodo personalizado para realizar consultas SQL en Jupyter</h2>

Vamos a utilizar ``JupySQL`` para realizar las consultas.

``JupySQL`` se desarrolló sobre iPython-SQL y su propósito es conectarse a BDs de varios tipos y ejecutar queries en Jupyter.

Esta será nuestra herramienta: ``JupySQLQuery`` y la definiremos como una subclase de la clase `BaseComponent` en Haystack.

También crearemos una indicación con instrucciones detalladas sobre cómo debe responder el agente a diferentes situaciones.




  <img src="diagrams/JupySQL-agent.png" width="500px"/>


In [ ]:
from haystack.nodes.base import BaseComponent

class JupySQLQuery(BaseComponent):
    outgoing_edges = 1
    
    def __init__(self):
        %reload_ext sql
        %sql duckdb:///bank.duck.db

    def run(self, query: str):
        result = %sql {{query}}
        output = {
            "results":  f"{result}",
            "query": query,
            
        }
        return output

    def run_batch(self, queries: list):
        results = []
        for query in queries:
            result = %sql {query}
            output = {
                "results":  f"{result}",
                "query": query,
            }
            results.append(output)
        return results

    
jupy_sql_query = JupySQLQuery()

In [ ]:
from haystack.agents import Tool
from haystack.nodes import PromptNode
from jupysqlagent import sql_agent_prompt
from haystack.agents import Agent, Tool

jupy_sql_query_tool = Tool(name="JupySQL_Query", 
                           pipeline_or_node=jupy_sql_query, 
                           description="""This tool is useful for consuming SQL queries \
                                        and responds with the result""")


In [ ]:
# Get the API key
openai_api_key = os.environ.get("openai-key")
chosen_model = "gpt-4"


# Define a prompt node that uses the GPT-4 model
prompt_node = PromptNode(model_name_or_path=chosen_model, 
                         api_key=openai_api_key, 
                         stop_words=["Observation:"], 
                         max_length=1000)

# Define the agent
agent = Agent(prompt_node=prompt_node, 
              prompt_template=sql_agent_prompt)

agent.add_tool(jupy_sql_query_tool)

In [ ]:
result = agent.run("How many records are there")


In [ ]:
result = agent.run("How many unique levels of education are there")


In [ ]:
result = agent.run("How many unique levels of education are there, \
                    what is the average employee age? ")


<h3 align='center'>Pros y contras de LangChain</h3>

**Pros**

1. Fácil de empezar a usar
2. Se mapea fácilmente al punto final de completado de chat de la API de OpenAI
3. Puede conectarse fácilmente a una variedad de aplicaciones basadas en tu definición de función


**Contras**

1. Preocupaciones de seguridad
2. Evaluación de resultados
3. Despliegue
4. La integración con LLMs abiertos y alojados parece estar en etapas iniciales





<h3 align='center'>Pros y contras de Haystack</h3>

**Pros**

1. Marco establecido con un enfoque en aplicaciones de NLP listas para producción
2. Constantemente se adapta a nuevos cambios y construye sobre su marco de trabajo
3. Amigable para el lanzamiento a producción
4. Ofrece soluciones para tus documentos personalizados y acceso a una variedad de sabores de base de datos
5. Ofrece plantillas de indicaciones
6. NLP pipelines listas para ser usadas

**Contras**

1. Curva de aprendizaje más pronunciada
2. La opción de lanzamiento a producción actual es la API REST, pero otras opciones actualmente no están disponibles
3. Limitaciones en los tipos de archivos que puede manejar (actualmente no se admiten PDF y markdown)
4. Enfoque más estrecho cuando se trata de los tipos de agentes que admite (aunque puedes crear agentes personalizados, a través de nodos personalizados)

<h1 align='center'>Reflexiones finales</h1>

* La ingeniería de indicaciones (prompt engineering) comienza con un proyecto bien definido y una elección clara de la arquitectura del transformador
* Las indicaciones son generalmente el primer paso al usar un LLM
* Las indicaciones a través de la API de OpenAI proporcionan una solución rápida para prototipos, pero tiene limitaciones cuando se trata de datos/documentos privados
* Las indicaciones a los LLMs de código abierto requieren la comprensión de la arquitectura del transformador y la apertura para afinar
* Exploramos dos marcos de trabajo de código abierto que te permiten aumentar la funcionalidad de los LLMs a través de agentes
* LangChain aborda a los agentes a través de componentes y cadenas
* Haystack aborda a los agentes en términos de expansión de la funcionalidad de nodos de indicaciones, pipelines y una tienda de documentos.